### Extralife 'Market the price of your life & the value of your dead'

**Objetive**

*We want a system that emulates the behaviour of an insurance company that keeps track of the most important metrics relevant to the system and KPIs that show the wellbeing of the system*

The Metrics for us to track are the following:

    -Size of the Risk-pool
    -Size of assured Sum to be payed
    -Amount of clients (Minted NFTs)
    -Price of the native coin
    -Income model
        -Minting income
        -Trade income
        -Fund income
        
The KPIs for us to track are the following:

    -Acidity ratio (Liabilities / Size of the Risk Pool)

### 0. Dependencies

In [1]:
import pandas as pd
import numpy as np
from random import normalvariate
import plotly.express as px

from cadCAD.configuration.utils import config_sim
from cadCAD.configuration import Experiment
from cadCAD.engine import ExecutionContext, Executor

### 1. State Variables

In [2]:
initial_state = {
    'risk_pool': 250000,
    'minted_nfts': 100,
    'size_of_assured_sum' : 1000 #Considering that each client payed 1 unit and receives 10 as assured sum
    #'native_coin_price'
}

### 2. System Parameters

**The system parameters we need to define are:**

* Adoption Rate: `bonding_rate` with value `.2`
* Death Rate: `death_rate` with value `.1`
* Native Coin Fluctuation Rate `coin_fluctuation` with value `.05`

#### 1 Time stamp = 1 Month

In [17]:
system_params = {
    'bonding_rate': [.001],
    'death_rate': [.001],
    #'coin_fluctuation': .05, # This parameter expands the model
    }

### 3. Policy Functions (Signals) (Bahaviours)

In [18]:
def new_clients (params, substep, state_history, previous_state):
    
    # Parameters & variables
    
    w = previous_state['minted_nfts']
    value = w * params['bonding_rate']#Randomness needs to be added here
    
    # Logic # We limit the amount of new clients to be realistic 
    
    #if new >= 100000:
    #    value = 100000
   # else:
     #   value = new

    # Output
    return {'new_clients': value}



In [19]:
def dead_clients (params, substep, state_history, previous_state):
    
    # Parameters & variables
    
    v = previous_state['minted_nfts']
    value = v * params['death_rate']#Randomness needed 
    
    # Output
    return {'remove_clients': value}

***Considerations
#New premiums is a substep (THE SIGNAL IS THE AMOUNT OF NEW CLIENTS)
#New assured sum debt is a substep (THE SIGNAL IS THE AMOUNT OF NEW CLIENTS) 

### 4. State Update Functions

In [20]:
def s_risk_pool (params, substep, state_history, previous_state, policy_input):
    
    # Parameters & variables
    current_size = previous_state['risk_pool'] + policy_input['new_clients']
    
    # Logic
    
    # Output
    return ('risk_pool', current_size)

In [21]:
def s_minted_nfts (params, substep, state_history, previous_state, policy_input):
    
    # Parameters & variables
    new_nfts = policy_input['new_clients'] + policy_input['new_clients']
    
    # Logic
    
    # Output
    return ('minted_nfts', new_nfts)


In [22]:
def s_size_of_assured_sum (params, substep, state_history, previous_state, policy_input):
    
    # Parameters & variables
    current_size = previous_state['risk_pool']
    current_assured_size = current_size*10 + (policy_input['new_clients'])*10
    
    # Logic
    #new_co2 = max(current_co2 + co2_change, 0)
    
    # Output
    return ('size_of_assured_sum', current_assured_size)


### 5. Partial State Update Blocks

In [23]:
partial_state_update_blocks = [
    {
        'policies': {
            'new_clients': new_clients,
            'dead_clients': dead_clients
        },
        'variables': {
            'risk_pool' : s_risk_pool,
            'minted_nfts' : s_minted_nfts,
            'size_of_assured_sum' :  s_size_of_assured_sum
            
            #'added_risk': s_risk_pool,
            #'added_minted_nfts': s_minted_nfts,
            #'added_size_of_assured_sum' : s_size_of_assured_sum
        }
    }
]

### 6. Configuration

In [24]:
MONTE_CARLO_RUNS = 1 #50 is a lot now
SIMULATION_TIMESTEPS = 100

sim_config = config_sim(
    {
        'N': MONTE_CARLO_RUNS,
        'T': range(SIMULATION_TIMESTEPS),
        'M': system_params,
    }
)

from cadCAD import configs
del configs[:] # Clear any prior configs

experiment = Experiment()
experiment.append_configs(
    sim_configs=sim_config,
    initial_state=initial_state,
    partial_state_update_blocks=partial_state_update_blocks
)

### 7. Execution

In [25]:
exec_context = ExecutionContext()

simulation = Executor(exec_context=exec_context, configs=configs)
raw_result, tensor_field, sessions = simulation.execute()


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

Execution Mode: local_proc
Configuration Count: 1
Dimensions of the first simulation: (Timesteps, Params, Runs, Vars) = (100, 2, 1, 3)
Execution Method: local_simulations
SimIDs   : [0]
SubsetIDs: [0]
Ns       : [0]
ExpIDs   : [0]
Execution Mode: single_threaded
Total execution time: 0.04s


# 8. Simulation Output Preparation
> The simulation results are returned as a list of Python dictionaries, which we then convert to a Pandas dataframe.

In [26]:
simulation_result = pd.DataFrame(raw_result)

In [27]:
simulation_result

,risk_pool,minted_nfts,size_of_assured_sum,simulation,subset,run,substep,timestep
0,250000.0000,1.000000e+02,1.000000e+03,0,0,1,0,0
1,250000.1000,2.000000e-01,2.500001e+06,0,0,1,1,1
2,250000.1002,4.000000e-04,2.500001e+06,0,0,1,1,2
3,250000.1002,8.000000e-07,2.500001e+06,0,0,1,1,3
4,250000.1002,1.600000e-09,2.500001e+06,0,0,1,1,4
...,...,...,...,...,...,...,...,...
96,250000.1002,7.922816e-258,2.500001e+06,0,0,1,1,96
97,250000.1002,1.584563e-260,2.500001e+06,0,0,1,1,97
98,250000.1002,3.169127e-263,2.500001e+06,0,0,1,1,98
99,250000.1002,6.338253e-266,2.500001e+06,0,0,1,1,99


In [28]:
df = simulation_result.copy()
df1 = df[df.simulation == 0]
df2 = df1.loc[:,['timestep','risk_pool','minted_nfts','size_of_assured_sum']]

In [29]:
df2

,timestep,risk_pool,minted_nfts,size_of_assured_sum
0,0,250000.0000,1.000000e+02,1.000000e+03
1,1,250000.1000,2.000000e-01,2.500001e+06
2,2,250000.1002,4.000000e-04,2.500001e+06
3,3,250000.1002,8.000000e-07,2.500001e+06
4,4,250000.1002,1.600000e-09,2.500001e+06
...,...,...,...,...
96,96,250000.1002,7.922816e-258,2.500001e+06
97,97,250000.1002,1.584563e-260,2.500001e+06
98,98,250000.1002,3.169127e-263,2.500001e+06
99,99,250000.1002,6.338253e-266,2.500001e+06


In [30]:
plot = px.line(df2,x='timestep', y=['risk_pool','minted_nfts','size_of_assured_sum'],title='Insurance Performance')
plot